In [1]:
import pandas as pd
data_18037=pd.read_csv('18037.csv')
data_18043=pd.read_csv('18043_merged.csv')
data_constituency=pd.read_csv('merged_constituencies_data.csv')

/var/tmp/ipykernel_3966/1931699567.py:2: DtypeWarning: Columns (9,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data_18037=pd.read_csv('18037.csv')


In [2]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, select, Float, UniqueConstraint, Sequence, BigInteger, ForeignKey, Boolean

In [3]:
engine = create_engine("sqlite:///elections.db")
metadata_obj = MetaData()

In [4]:
metadata_obj.clear()

In [5]:
states = Table(
    'states',
    metadata_obj,
    Column("id", Integer, Sequence('state_id_seq'), primary_key=True),
    Column("name", String(32)),
    Column("is_active", Boolean),
)
# Create the tables in the database
metadata_obj.create_all(engine)

In [6]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [7]:
from sqlalchemy import create_engine, insert, select
from sqlalchemy.exc import IntegrityError

In [8]:
Active_States_Data=["Andhra Pradesh","Arunachal Pradesh","Assam",
"Bihar","Chhattisgarh","Goa","Gujarat","Haryana","Himachal Pradesh","Jharkhand",
"Karnataka","Kerala","Madhya Pradesh","Maharashtra","Manipur","Meghalaya","Mizoram",
"Nagaland","Odisha","Punjab","Rajasthan","Sikkim","Tamil Nadu","Telangana","Tripura",
"Uttar Pradesh","Uttarakhand","West Bengal"]
Active_Union_Territories_Data=["Andaman and Nicobar Islands","Chandigarh","Dadra and Nagar Haveli","Daman and Diu","Delhi",
"Jammu and Kashmir","Ladakh","Lakshadweep","Puducherry"]

In [9]:
#Normalize and insert the data into the states table
states_data=data_18037[["state"]].drop_duplicates()

# Iterate over the rows and insert data into the states table
for index, row in states_data.iterrows():
    # Check if the state already exists
    existing_state = session.query(states).filter_by(
        name=row["state"].strip()
    ).first()
    
    if existing_state:
        # If the state already exists, skip it
        continue
    
    else:
        if row["state"].strip() in Active_States_Data or row["state"].strip() in Active_Union_Territories_Data:
            active=True
            # If the state doesn't exist, create a new one
            # Insert the new state into the database
            session.execute(
                states.insert(),
                {
                    'name': row['state'],
                    'is_active': active
                }
            )

        else:
            active=False
            # If the state doesn't exist, create a new one
            # Insert the new state into the database
            session.execute(
                states.insert(),
                {
                    'name': row['state'],
                    'is_active': active
                }
            )
        
# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [10]:
# Query the states table
states_data = session.query(states).all()

# Convert the query result to a Pandas DataFrame
states_data_df = pd.DataFrame(states_data)

# Display the DataFrame
display(states_data_df)

,id,name,is_active
0,1,West Bengal,True
1,2,Uttarakhand,True
2,3,Uttar Pradesh,True
3,4,Tripura,True
4,5,Telangana,True
5,6,Tamil Nadu,True
6,7,Sikkim,True
7,8,Rajasthan,True
8,9,Punjab,True
9,10,Puducherry,True


In [11]:
#elctions table 
elections = Table(
    'elections',
    metadata_obj,
    Column("id", Integer, Sequence('election_id_seq'), primary_key=True),
    Column("type", String(32)),
    Column("year", Integer),
    Column("number", Integer),
)
metadata_obj.create_all(engine)

In [12]:
# Create a mapping dictionary for the Lok Sabha number
lok_sabha_mapping = {
    "First": 1,
    "Second": 2,
    "Third": 3,
    "Fourth": 4,
    "Fifth": 5,
    "Sixth": 6,
    "Seventh": 7,
    "Eighth": 8,
    "Ninth": 9,
    "Tenth": 10,
    "Eleventh": 11,
    "Twelfth": 12,
    "Thirteenth": 13,
    "Fourteenth": 14,
    "Fifteenth": 15,
    "Sixteenth": 16,
    "Seventeenth": 17
}

In [13]:
# Normalize and insert the data into the elections table
elections_data=data_18037[["year", "lok_sabha_number"]].drop_duplicates()
# Iterate over the rows and insert data into the elections table
for index, row in elections_data.iterrows():
    # Check if the election already exists
    existing_election = session.query(elections).filter_by(
        year=row["year"],
        number=lok_sabha_mapping[row["lok_sabha_number"]]
    ).first()
    
    if existing_election:
        # If the election already exists, skip it
        continue
    
    else:
        # If the election doesn't exist, create a new one
        # Insert the new election into the database
        session.execute(
            elections.insert(),
            {
                'type': "Lok Sabha",
                'year': row['year'],
                'number': lok_sabha_mapping[row["lok_sabha_number"]]
            }
        )
        
# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [14]:
# Query the constituencies table
elect_data = session.query(elections).all()

# Convert the query result to a Pandas DataFrame
elect_data_df = pd.DataFrame(elect_data)

# Display the DataFrame
display(elect_data_df)

,id,type,year,number
0,1,Lok Sabha,2019,17
1,2,Lok Sabha,2014,16
2,3,Lok Sabha,2009,15
3,4,Lok Sabha,2004,14
4,5,Lok Sabha,1999,13
5,6,Lok Sabha,1998,12
6,7,Lok Sabha,1996,11
7,8,Lok Sabha,1992,10
8,9,Lok Sabha,1991,10
9,10,Lok Sabha,1989,9


In [15]:
#Constituencies table
constituencies = Table(
    'constituencies',
    metadata_obj,
    Column("id", Integer, Sequence('constituency_id_seq'), primary_key=True),
    Column("election_id", Integer, ForeignKey('elections.id')),
    Column("state_id", Integer, ForeignKey('states.id')),
    Column("name", String(32)),
    Column("number", Integer),
    Column("type", String(32)),
    Column("is_active", Boolean),
    #extend_existing=True,
)
# Create the tables in the database
metadata_obj.create_all(engine)

In [16]:
# Normalize and insert the data into the constituencies table
#constituencies_data = data_18037[["year","lok_sabha_number","state","constituency_name","constituency_number","constituency_type"]].drop_duplicates()
constituencies_data = data_constituency[["year","lok_sabha_number","state","constituency_name","constituency_number","constituency_type"]]

#Finding the max year
max_year = data_constituency["year"].drop_duplicates().max()

# Iterate over the rows and insert data into the constituencies table
for index, row in constituencies_data.iterrows():
    # Check if the constituency already exists
    """existing_constituency = session.query(constituencies).filter_by(
        election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id,
        state_id=session.query(states).filter_by(name=row["state"].strip()).first().id,
        name=row["constituency_name"].strip(),
        number=row["constituency_number"],
        type=row["constituency_type"]
    ).first()

    if existing_constituency:
        # If the constituency exists, update its active status
        continue
        """

    # Get the election_id and state_id
    election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id
    state_id = session.query(states).filter_by(name=row["state"].strip()).first().id
    if row["year"] == max_year:
        constituency_active = True 
    else:
        #constituency as inactive
        constituency_active = False

    # If the constituency doesn't exist, create a new one
    # Insert the new constituency into the database
    session.execute(
        constituencies.insert(),
        {
            'election_id' : election_id,
            'state_id': state_id,
            'name': row['constituency_name'],
            'number': row["constituency_number"],
            'type' : row["constituency_type"],
            'is_active': constituency_active,
        }
    )

# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [17]:
# Query the constituencies table
const_data = session.query(constituencies).all()

# Convert the query result to a Pandas DataFrame
const_data_df = pd.DataFrame(const_data)

# Display the DataFrame
display(const_data_df)

,id,election_id,state_id,name,number,type,is_active
0,1,1,1,Birbhum,42,General,True
1,2,1,1,Bholpur,41,Scheduled Caste,True
2,3,1,1,Asansol,40,General,True
3,4,1,1,Bardhaman Durgapur,39,General,True
4,5,1,1,Bardhaman Purba,38,Scheduled Caste,True
...,...,...,...,...,...,...,...
8794,8795,19,33,Goalpara Garo Hills,3,General,False
8795,8796,19,33,Autonomous District,2,General,False
8796,8797,19,33,Cachar Lushal Hills,1,General,False
8797,8798,19,51,Ajmer South,2,General,False


In [18]:
#election_stats table
election_stats = Table(
    'election_stats',
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('category', String(255)),
    Column('gender', String(255)),
    Column('value', Integer),
    #Column('state_id', Integer),
    Column('constituency_id', Integer, ForeignKey('constituencies.id')),
    #Column('election_id', Integer, ForeignKey('elections.id')),
    #extend_existing=True
)

# Create the table in the database
metadata_obj.create_all(engine)

In [19]:
#Normalize and insert the data into the election_stats table
election_stats_data=data_18043[["year","lok_sabha_number","state","constituency_type","constituency_number","constituency_name","category","gender","value"]].drop_duplicates()

# Iterate over the rows and insert data into the election_stats table
for index, row in election_stats_data.iterrows():
    # Check if the election_stats already exists
    election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id
    state_id=session.query(states).filter_by(name=row["state"].strip()).first().id
    """existing_election_stats = session.query(election_stats).filter_by(
        constituency_id=session.query(constituencies).filter_by(election_id=election_id,state_id=state_id,name=row["constituency_name"], number=row["constituency_number"]).first().id,
        category=row["category"],
        gender=row["gender"],
        value=row["value"]
    ).first()

    if existing_election_stats:
        # If the election_stats already exists, skip it
        continue
    """
    # If the election_stats doesn't exist, create a new one
    # Insert the new election_stats into the database
    session.execute(
        election_stats.insert(),
        {
            'category': row['category'],
            'gender':row['gender'],
            'value':row['value'],
            #'state_id': session.query(states).filter_by(name=row["state"].strip()).first().id,
            'constituency_id': session.query(constituencies).filter_by(election_id=election_id,state_id=state_id, name=row["constituency_name"], number=row["constituency_number"]).first().id,
            #'election_id': session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id,
        }
    )

# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [20]:
# Query the election_stats table
election_stat_data = session.query(election_stats).all()

# Convert the query result to a Pandas DataFrame
election_stat_data_df = pd.DataFrame(election_stat_data)

# Display the DataFrame
display(election_stat_data_df)

,id,category,gender,value,constituency_id
0,1,Contesting Candidates,Female,1,34
1,2,Contesting Candidates,Male,10,34
2,3,Contesting Candidates,Total,11,34
3,4,Forfeited Deposits,Male,8,34
4,5,Forfeited Deposits,Female,0,34
...,...,...,...,...,...
87830,87831,Nominations Rejected,Female,0,6992
87831,87832,Nominations Rejected,Total,0,6992
87832,87833,Nominations Withdrawn,Male,5,6992
87833,87834,Nominations Withdrawn,Female,0,6992


In [21]:
#parties table
parties = Table(
    'parties',
    metadata_obj,
    Column("id", Integer, Sequence('party_id_seq'), primary_key=True),
    Column("name", String(32)),
    Column("abbreviation", String(32)),
    Column("type", String(32)),
    #Column("symbol", String(32)),
)
metadata_obj.create_all(engine)

In [22]:
# Normalize and insert the data into the parties table
parties_data=data_18037[["party_name", "party_abbreviation", "party_type"]].drop_duplicates()

# Iterate over the rows and insert data into the parties table
for index, row in parties_data.iterrows():
    # Check if the party already exists
    existing_party = session.query(parties).filter_by(
        name=row["party_name"],
        abbreviation=row["party_abbreviation"],
        type=row["party_type"],
        #symbol=row["party_symbol"]
    ).first()
    
    if existing_party:
        # If the party already exists, skip it
        continue
    
    else:
        # If the party doesn't exist, create a new one
        # Insert the new party into the database
        session.execute(
            parties.insert(),
            {
                'name': row['party_name'],
                'abbreviation': row['party_abbreviation'],
                'type': row['party_type'],
                #'symbol': row['party_symbol']
            }
        )
        
# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [23]:
# Query the parties table
party_data = session.query(parties).all()

# Convert the query result to a Pandas DataFrame
party_data_df = pd.DataFrame(party_data)

# Display the DataFrame
display(party_data_df)

,id,name,abbreviation,type
0,1,Indian National Congress,INC,National Party
1,2,All India Trinamool Congress,AITC,National Party
2,3,Bharatiya Janata Party,BJP,National Party
3,4,Communist Party of India(Marxist),CPM,National Party
4,5,Bahujan Samaj Party,BSP,National Party
...,...,...,...,...
1682,1683,All Peoples Party,APP,State Party
1683,1684,Tribal Sangha,TS,State Party
1684,1685,Khasi-Jaintia Durbar,KJD,State Party
1685,1686,Hill People Party,HPP,State Party


In [24]:
#candidates table
candidates = Table(
    'candidates',
    metadata_obj,
    Column("id", Integer, Sequence('candidate_id_seq'), primary_key=True),
    Column("name", String(32)),
    Column("gender", String(32)),
    Column("age", Integer),
    Column("category", String(32)),
    Column("constituency_id", Integer, ForeignKey('constituencies.id')),
    #Column("election_id", Integer, ForeignKey('elections.id')),
    Column("party_id", Integer, ForeignKey('parties.id')),
)
metadata_obj.create_all(engine)

In [25]:
# Normalize and insert the data into the candidates table
candidates_data=data_18037[["candidate_name", "gender", "age", "candidate_category","state", "constituency_name", "constituency_number", "year", "lok_sabha_number", "party_name", "party_abbreviation", "party_type"]].drop_duplicates()

# Iterate over the rows and insert data into the candidates table
for index, row in candidates_data.iterrows():
    # Check if the candidate already exists
    election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id
    state_id=session.query(states).filter_by(name=row["state"].strip()).first().id
    existing_candidate = session.query(candidates).filter_by(
        name=row["candidate_name"].strip(),
        age=row["age"],
        gender=row["gender"],
        category=row["candidate_category"],
        constituency_id=session.query(constituencies).filter_by(election_id=election_id, state_id=state_id, name=row["constituency_name"], number=row["constituency_number"]).first().id,
        #election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id,
        party_id=session.query(parties).filter_by(name=row["party_name"], abbreviation=row["party_abbreviation"], type=row["party_type"]).first().id if session.query(parties).filter_by(name=row["party_name"], abbreviation=row["party_abbreviation"], type=row["party_type"]).first() else None,
    ).first()

    if existing_candidate:
        # If the candidate already exists, skip it
        continue

    # If the candidate doesn't exist, create a new one
    # Insert the new candidate into the database
    session.execute(
        candidates.insert(),
        {
            'name': row['candidate_name'],
            'gender': row['gender'],
            'age': row['age'],
            'category': row['candidate_category'],
            'constituency_id': session.query(constituencies).filter_by(election_id=election_id, state_id=state_id, name=row["constituency_name"], number=row["constituency_number"]).first().id,
            #'election_id': session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id,
            'party_id': session.query(parties).filter_by(name=row["party_name"], abbreviation=row["party_abbreviation"], type=row["party_type"]).first().id if session.query(parties).filter_by(name=row["party_name"], abbreviation=row["party_abbreviation"], type=row["party_type"]).first() else None,
        }
    )

# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [26]:
# Query the candidates table
candidate_data = session.query(candidates).all()

# Convert the query result to a Pandas DataFrame
candidate_data_df = pd.DataFrame(candidate_data)

# Display the DataFrame
display(candidate_data_df)

,id,name,gender,age,category,constituency_id,party_id
0,1,Abhijit Mukherjee,Male,59.0,General,34,1.0
1,2,Khalilur Rahaman,Male,58.0,General,34,2.0
2,3,Mafuja Khatun,Female,48.0,General,34,3.0
3,4,Md. Zulfikar Ali,Male,53.0,General,34,4.0
4,5,Shamimul Islam,Male,42.0,General,34,5.0
...,...,...,...,...,...,...,...
91902,91903,Jawala Prashad,None,NaN,None,8799,1.0
91903,91904,Chand Karan,None,NaN,None,8799,1636.0
91904,91905,Dino Mal,None,NaN,None,8799,1687.0
91905,91906,Bajoria Badridas,None,NaN,None,8799,10.0


In [27]:
##election_results table
election_results = Table(
    'election_results',
    metadata_obj,
    Column("id", Integer, Sequence('election_result_id_seq'), primary_key=True),
    Column("candidate_id", Integer, ForeignKey('candidates.id')),
    #Column("constituency_id", Integer, ForeignKey('constituencies.id')),
    Column("general_votes_secured", Integer),
    Column("postal_votes_secured", Integer),
    Column("total_votes_secured", Integer),
    Column("percent_votes_secured_by_valid_votes", Float),
    Column("candidate_position", Integer),
    #extend_existing=True,
)

# Create the tables in the database
metadata_obj.create_all(engine)

In [28]:
#Normalize and insert the data into the election_results table
election_results_data=data_18037[["candidate_name", "gender", "age", "candidate_category","state", "constituency_name", "constituency_number","constituency_type", "year", "lok_sabha_number", "party_name", "party_abbreviation", "party_type","general_votes_secured","postal_votes_secured","total_votes_secured","percent_votes_secured_by_valid_votes","position"]].drop_duplicates()

# Iterate over the rows and insert data into the election_results table
for index, row in election_results_data.iterrows():
    # Check if the election_result already exist
    election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id
    state_id=session.query(states).filter_by(name=row["state"].strip()).first().id
    candidate = session.query(candidates).filter_by(
        #election_id=session.query(elections).filter_by(year=row["year"], number=lok_sabha_mapping[row["lok_sabha_number"]]).first().id,
        constituency_id=session.query(constituencies).filter_by(election_id=election_id, state_id=state_id, name=row["constituency_name"], number=row["constituency_number"]).first().id,
        name=row["candidate_name"].strip(),
        #age=row["age"],
        #gender=row["gender"],
        #category=row["candidate_category"]
    ).first()
    
    """constituency = session.query(constituencies).filter_by(
        constituency_id=session.query(constituencies).filter_by(name=row["constituency_name"], number=row["constituency_number"]).first().id,
    ).first()"""

    """existing_election_result = session.query(election_results).filter_by(
        candidate_id=candidate.id,
        election_constituencies_id=election_constituency.id,
        general_votes_secured=row["general_votes_secured"],
        postal_votes_secured=row["postal_votes_secured"],
        total_votes_secured=row["total_votes_secured"],
        percent_votes_secured_by_valid_votes=row["percent_votes_secured_by_valid_votes"],
        candidate_position=row["position"]
    ).first()

    if existing_election_result:
        # If the election_result already exists, skip it
        continue"""
    
    # Change 'Uncontested' value to 0
    if row["percent_votes_secured_by_valid_votes"] == "Uncontested":
        row["percent_votes_secured_by_valid_votes"] = 0
    # If the election_result doesn't exist, create a new one
    # Insert the new election_result into the database
    session.execute(
        election_results.insert(),
        {
            'candidate_id': candidate.id,
            #'constituency_id': constituency.id,
            'general_votes_secured': row["general_votes_secured"],
            'postal_votes_secured': row["postal_votes_secured"],
            'total_votes_secured': row["total_votes_secured"],
            'percent_votes_secured_by_valid_votes': row["percent_votes_secured_by_valid_votes"],
            'candidate_position': row["position"]
        }
    )

# Commit the changes to the database
try:
    session.commit()
except IntegrityError:
    session.rollback()

# Close the session
session.close()

In [29]:
# Query the election_results table
election_result_data = session.query(election_results).all()

# Convert the query result to a Pandas DataFrame
election_result_df = pd.DataFrame(election_result_data)

# Display the DataFrame
display(election_result_df)

,id,candidate_id,general_votes_secured,postal_votes_secured,total_votes_secured,percent_votes_secured_by_valid_votes,candidate_position
0,1,1,255554.0,282.0,255836.0,19.61,3
1,2,2,562518.0,320.0,562838.0,43.14,1
2,3,3,316147.0,909.0,317056.0,24.30,2
3,4,4,95450.0,51.0,95501.0,7.32,4
4,5,5,3497.0,1.0,3498.0,0.27,11
...,...,...,...,...,...,...,...
91953,91954,91903,NaN,NaN,46679.0,48.04,1
91954,91955,91904,NaN,NaN,28990.0,29.84,2
91955,91956,91905,NaN,NaN,10778.0,11.09,3
91956,91957,91906,NaN,NaN,6153.0,6.33,4


In [30]:
# Defining the table names and summaries for the above tables
table_infos = [
    {'table_name': 'states', 'table_summary': 'Stores information about states, including their unique ID, name, and active status.'},
    {'table_name': 'elections', 'table_summary': 'Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'},
    {'table_name': 'constituencies', 'table_summary': 'Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'},
    {'table_name': 'parties', 'table_summary': 'Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'},
    {'table_name': 'candidates', 'table_summary': 'Contains details about candidates contesting in elections and their respective constituencies. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'},
    {'table_name': 'election_results', 'table_summary': 'Records the election results based on the candidates data, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate\'s position in the election results.'},
    {'table_name': 'election_stats', 'table_summary': 'Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'}
]

In [31]:
%pip install llama-index-llms-openai

Note: you may need to restart the kernel to use updated packages.


In [32]:
from sqlalchemy import create_engine, MetaData
from llama_index.core import SQLDatabase
from llama_index.core.objects import SQLTableNodeMapping, ObjectIndex, SQLTableSchema
from llama_index.core.retrievers import SQLRetriever
from llama_index.core.query_pipeline import QueryPipeline as QP, Link, InputComponent
from llama_index.llms.openai import OpenAI

In [33]:
import openai
import os

openai.api_key = "API_KEY"
os.environ["OPENAI_API_KEY"] = "API_KEY"

In [34]:
%pip install llama-index-embeddings-openai

Note: you may need to restart the kernel to use updated packages.


In [35]:
from llama_index.core.objects import (
SQLTableNodeMapping,
ObjectIndex,
SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
SQLTableSchema(table_name=info['table_name'], context_str=info['table_summary'])
for info in table_infos
] # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
table_schema_objs,
table_node_mapping,
VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=6)

In [36]:
#Text-to-SQL Prompt + Output Parser
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [37]:
#Response Synthesis Prompt
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [38]:
llm = OpenAI(model="gpt-4-turbo")
#llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
import os
from pathlib import Path
from typing import Dict

def index_all_tables(
    sql_database: SQLDatabase, table_index: str = "table_index", tables_to_index=None, chunk_size=256
) -> Dict[str, VectorStoreIndex]:
    if not Path(table_index).exists():
        os.makedirs(table_index, exist_ok=True)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        # Skip tables that are not in the list of tables to index
        if tables_to_index is not None and table_name not in tables_to_index:
            continue

        print(f"Indexing rows in table: {table_name}")
        table_index_path = f"{table_index}/{table_name}"
        if not os.path.exists(table_index_path):
            # We will create the VectorStoreIndex after fetching the rows
            nodes = []

            # Process the table in chunks
            with engine.connect() as conn:
                offset = 0
                while True:
                    query = text(f'SELECT * FROM "{table_name}" LIMIT :limit OFFSET :offset')
                    cursor = conn.execute(query, {"limit": chunk_size, "offset": offset})
                    result = cursor.fetchall()
                    if not result:
                        break

                    # Create nodes for each row in the chunk
                    nodes.extend([TextNode(text=str(t)) for t in result])

                    offset += chunk_size
                    
            #Basically, it iterates all the rows from tables first it takes the 256 rows and add it to the node list.
            #Continuously it is performed till when their is no rows in the table. After that then performs the indexing by adding the all the nodes to the vectorstoreindex.
            # Now we create the VectorStoreIndex with the nodes
            index = VectorStoreIndex(nodes)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(persist_dir=f"{table_index}/{table_name}")
            # load index
            index = load_index_from_storage(storage_context, index_id="vector_index")
            
        vector_index_dict[table_name] = index

    return vector_index_dict

# Specify which tables you want to index
tables_to_index = ['states','constituencies','elections','parties','candidates','election_results','election_stats']

# Index only the specified tables with chunk size. 
vector_index_dict = index_all_tables(sql_database, tables_to_index=tables_to_index,chunk_size=256)

Indexing rows in table: candidates
Indexing rows in table: constituencies
Indexing rows in table: election_results
Indexing rows in table: election_stats
Indexing rows in table: elections
Indexing rows in table: parties
Indexing rows in table: states


In [40]:
print(vector_index_dict)

{'candidates': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f74134d8ac0>, 'constituencies': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f74134d8c40>, 'election_results': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f7412ca8e20>, 'election_stats': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f7412cabe80>, 'elections': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f7412ca96c0>, 'parties': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f7412caa890>, 'states': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7f7412caa560>}


In [41]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
logging.getLogger().handlers = []

In [42]:
#Debugging
from llama_index.core.query_pipeline import query
from typing import Any

def print_debug_input_new(
    module_key: str,
    input: Dict[str, Any],
    val_str_len: int = 200000000,
) -> None:
    """Print debug input."""
    output = f"> Running module {module_key} with input: \n"
    for key, value in input.items():
        # stringify and truncate output
        val_str = (
            str(value)[:val_str_len] + "..."
            if len(str(value)) > val_str_len
            else str(value)
        )
        output += f"{key}: {val_str}\n"

    query.print_text(output + "\n", color="llama_lavender")

query.print_debug_input = print_debug_input_new

In [43]:
from llama_index.core.retrievers import SQLRetriever
from typing import List, Dict, Tuple, Any
from llama_index.core.query_pipeline import (
    FnComponent,
    QueryPipeline as QP,
    Link,
    InputComponent,
    AgentInputComponent,
    AgentFnComponent,
)
from llama_index.core import PromptTemplate, Response
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import (
    Task,
    AgentChatResponse,
    QueryPipelineAgentWorker,
    AgentRunner,
)
from llama_index.core.callbacks import CallbackManager

# Define the retry prompt
MAX_TRIES = 3
retry_prompt_str = """
You are trying to generate a proper SQL query given a user input.

If the SQL query is invalid or does not provide the correct results, modify it accordingly.

Input: {input}
Convo history (failed attempts): 
{convo_history}

Attempt {attempt_num}/{max_tries}: New SQL query:
"""
retry_prompt = PromptTemplate(retry_prompt_str)

In [44]:
# Define the agent input function
def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict:
    """Agent input function."""
    # Initialize conversation history and count
    if "convo_history" not in state:
        state["convo_history"] = []
        state["count"] = 0
    
    # Append user input to conversation history
    state["convo_history"].append(f"User: {task.input}")
    
    # Format conversation history as a string
    convo_history_str = "\n".join(state["convo_history"]) or "None"
    
    # Return input and conversation history
    return {"input": task.input, "convo_history": convo_history_str}

# Create the agent input component
agent_input_component = AgentInputComponent(fn=agent_input_fn)

In [45]:
# Define the output processor with validation
def agent_output_fn(
    task: Task, state: Dict[str, Any], output: Response
) -> Tuple[AgentChatResponse, bool]:
    """Agent output component with validation."""
    print(f"> Inferred SQL Query: {output.metadata['sql_query']}")
    print(f"> SQL Response: {str(output)}")
    
    # Append inferred SQL query and response to conversation history
    state["convo_history"].append(
        f"Assistant (inferred SQL query): {output.metadata['sql_query']}"
    )
    state["convo_history"].append(f"Assistant (response): {str(output)}")

    # Increment conversation count
    state["count"] += 1
    
    # Validate the SQL response using the LLM
    validate_prompt_str = """
    Given the user query, validate whether the SQL response is correct and answers the query.
    
    Answer with YES or NO.
    
    Query: {input}
    SQL Response: {sql_response}
    
    Result: 
    """
    validate_prompt = PromptTemplate(validate_prompt_str)
    validate_prompt_partial = validate_prompt.as_query_component(
        partial={
            "input": task.input,
            "sql_response": str(output),
        }
    )
    qp = QP(chain=[validate_prompt_partial, llm])
    validate_output = qp.run(input=task.input)

    # Check if the maximum number of tries has been reached or if the output is validated
    is_done = False
    if state["count"] >= MAX_TRIES:
        is_done = True
    if "YES" in validate_output.message.content:
        is_done = True

    # Return the agent response and completion status
    return AgentChatResponse(response=str(output)), is_done

# Create the agent output component with validation
agent_output_component = FnComponent(fn=agent_output_fn)

In [46]:
from llama_index.core.retrievers import SQLRetriever
sql_retriever = SQLRetriever(sql_database)

# Define the table output parser
def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # Append table info and optional context
        table_info = sql_database.get_single_table_info(table_schema_obj.table_name)
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # Retrieve relevant table rows using vector index
        vector_retriever = vector_index_dict[table_schema_obj.table_name].as_retriever(
            similarity_top_k=3
        )
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)

table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

In [91]:
def _run_component(self, **kwargs: Any) -> Dict:
    """Run component."""
    output = self.fn(**kwargs)
    # Ensure the output is a dictionary with exactly one key
    return {"output": output}

async def _arun_component(self, **kwargs: Any) -> Any:
    """Run component (async)."""
    if self.async_fn is None:
        return self._run_component(**kwargs)
    else:
        output = await self.async_fn(**kwargs)
        # Ensure the output is a dictionary with exactly one key
        return {"output": output}

In [95]:
from pydantic import BaseModel
from pydantic.fields import Field
from typing import Set,Tuple,List
class OutputKeys(BaseModel):
    """Output keys."""

    required_keys: Set[str] = Field(default_factory=set)

    @classmethod
    def from_keys(
        cls,
        required_keys: Set[str],
    ) -> "OutputKeys":
        """Create OutputKeys from tuple."""
        return cls(required_keys=required_keys)

    def validate(self, input_keys: Set[str]) -> None:
        """Validate input keys."""
        # validate that input keys exactly match required keys
        if input_keys != self.required_keys:
            raise ValueError(
                f"Input keys {input_keys} do not match required keys {self.required_keys}"
            )

In [110]:
from typing import Optional
def get_output(
    src_key: Optional[str],
    output_dict: Dict[str, Any],
) -> Any:
    """Add input to module deps inputs."""
    # get relevant output from link
    if src_key is None:
        # ensure that output_dict only has one key
        if len(output_dict) != 1:
            raise ValueError("Output dict must have exactly one key.")
        output = next(iter(output_dict.values()))
    else:
        output = output_dict[src_key]
    return output

In [111]:
# Define the query pipeline with retry agent
qp = QP(
    modules={
        "input": agent_input_component,
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "retry_prompt": retry_prompt,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
        "output_component": agent_output_component,
    },
    verbose=True,
)
# # Create the query pipeline with a single root module
# qp = QP(
#     modules={
#         "input": agent_input_component,
#         "table_retriever": obj_retriever,
#         "table_output_parser": table_parser_component,
#         "retry_prompt": retry_prompt,
#         "llm": llm,
#         "text2sql_prompt": text2sql_prompt,
#         "text2sql_llm": llm,
#         "sql_output_parser": sql_parser_component,
#         "sql_retriever": sql_retriever,
#         "response_synthesis_prompt": response_synthesis_prompt,
#         "response_synthesis_llm": llm,
#         "agent_output_component": agent_output_component,
#     },
#     verbose=True,
# )

In [112]:
# # Add links and chains for the query pipeline
qp.add_link("input", "retry_prompt", src_key="input", dest_key="input")
qp.add_link(
    "input", "retry_prompt", src_key="convo_history", dest_key="convo_history"
)
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")
qp.add_link("response_synthesis_llm", "output_component")


# # Add links and chains for the query pipeline
# qp.add_link("input", "retry_prompt", src_key="input", dest_key="input")
# qp.add_link("input", "retry_prompt", src_key="convo_history", dest_key="convo_history")
# qp.add_link("input", "table_retriever")
# qp.add_link("input", "table_output_parser", dest_key="query_str")
# qp.add_link("table_retriever", "table_output_parser", dest_key="table_schema_objs")
# qp.add_link("retry_prompt", "text2sql_prompt")
# qp.add_link("table_output_parser", "text2sql_prompt", src_key="schema", dest_key="schema")
# qp.add_chain(["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"])
# qp.add_link("sql_output_parser", "response_synthesis_prompt", src_key="sql_query", dest_key="sql_query")
# qp.add_link("sql_retriever", "response_synthesis_prompt", src_key="context_str", dest_key="context_str")
# qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
# qp.add_link("response_synthesis_prompt", "response_synthesis_llm")
# qp.add_link("response_synthesis_llm", "output_component")

In [113]:
# qp.add_chain(
#     [
#         "retry_prompt",
#         "text2sql_prompt",
#         "text2sql_llm",
#         "sql_output_parser",
#         "sql_retriever",
#         "response_synthesis_prompt",
#         "response_synthesis_llm",
#         "agent_output_component",
#     ]
# )
qp.add_chain(
    [
        "retry_prompt",
        "sql_output_parser",
        "sql_retriever",
        "response_synthesis_prompt",
        "response_synthesis_llm",
        "agent_output_component",
    ]
)

In [114]:
# Create the agent worker and runner
agent_worker = QueryPipelineAgentWorker(qp)
agent1 = AgentRunner(
    agent_worker, callback_manager=CallbackManager(), verbose=False
)

In [ ]:
response = agent1.chat(
    "How many Lok Sabha constituencies are there in Andhra Pradesh?"
)
print(str(response))

In [83]:
%pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [90]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)
net.show("agent_dag.html")

agent_dag.html


In [123]:
#defining the table output parser
#basically, it will generate the relevant table schemas along with the relevant rows per table schema.

from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=3)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

In [124]:
#defining the query pipeline
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [125]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [126]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
logging.getLogger().handlers = []

In [127]:
#Debugging
from llama_index.core.query_pipeline import query
from typing import Any

def print_debug_input_new(
    module_key: str,
    input: Dict[str, Any],
    val_str_len: int = 200000000,
) -> None:
    """Print debug input."""
    output = f"> Running module {module_key} with input: \n"
    for key, value in input.items():
        # stringify and truncate output
        val_str = (
            str(value)[:val_str_len] + "..."
            if len(str(value)) > val_str_len
            else str(value)
        )
        output += f"{key}: {val_str}\n"

    query.print_text(output + "\n", color="llama_lavender")

query.print_debug_input = print_debug_input_new

In [46]:
#Run with queries
response = qp.run(query="Who got the highest number of votes in 2019 in Hyderabad?")
print(str(response))

> Running module input with input: 
query: Who got the highest number of votes in 2019 in Hyderabad?

> Running module table_retriever with input: 
input: Who got the highest number of votes in 2019 in Hyderabad?

> Running module table_output_parser with input: 
query_str: Who got the highest number of votes in 2019 in Hyderabad?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the correspon

In [48]:
response = qp.run(query="How many Lok Sabha constituencies are there in Andhra Pradesh?")
print(str(response))

> Running module input with input: 
query: How many Lok Sabha constituencies are there in Andhra Pradesh?

> Running module table_retriever with input: 
input: How many Lok Sabha constituencies are there in Andhra Pradesh?

> Running module table_output_parser with input: 
query_str: How many Lok Sabha constituencies are there in Andhra Pradesh?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID,

In [49]:
response = qp.run(query="How many Lok Sabha constituencies are there in Andhra Pradesh in the year 2019?")
print(str(response))

> Running module input with input: 
query: How many Lok Sabha constituencies are there in Andhra Pradesh in the year 2019?

> Running module table_retriever with input: 
input: How many Lok Sabha constituencies are there in Andhra Pradesh in the year 2019?

> Running module table_output_parser with input: 
query_str: How many Lok Sabha constituencies are there in Andhra Pradesh in the year 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, ca

In [46]:
response = qp.run(query="How many ST candidates contested from BJP in Chhattisgarh in 2014?")
print(str(response))

> Running module input with input: 
query: How many ST candidates contested from BJP in Chhattisgarh in 2014?

> Running module table_retriever with input: 
input: How many ST candidates contested from BJP in Chhattisgarh in 2014?

> Running module table_output_parser with input: 
query_str: How many ST candidates contested from BJP in Chhattisgarh in 2014?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associate

In [47]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT COUNT(*) FROM candidates 
JOIN constituencies ON candidates.constituency_id = constituencies.id 
JOIN elections ON constituencies.election_id = elections.id 
JOIN parties ON candidates.party_id = parties.id 
WHERE candidates.category = 'Scheduled Tribe' AND parties.abbreviation = 'BJP' AND constituencies.state_id = (SELECT id FROM states WHERE name = 'Chhattisgarh') AND elections.year = 2014;
""")

# Fetch the result
result = cursor.fetchone()

# Print the result
print(result)

# Close the connection
conn.close()

(4,)


In [137]:
response = qp.run(query="What was the margin of victory for Bandi Sanjay Kumar? Margin of victory is the difference between the votes of first and second place candidates")
print(str(response))

> Running module input with input: 
query: What was the margin of victory for Bandi Sanjay Kumar? Margin of victory is the difference between the votes of first and second place candidates

> Running module table_retriever with input: 
input: What was the margin of victory for Bandi Sanjay Kumar? Margin of victory is the difference between the votes of first and second place candidates

> Running module table_output_parser with input: 
query_str: What was the margin of victory for Bandi Sanjay Kumar? Margin of victory is the difference between the votes of first and second place candidates
table_schema_objs: [SQLTableSchema(table_name='election_results', context_str="Records the election results based on the candidates data, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, inclu

In [ ]:
print(f'{response.sql_query}')

In [49]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT MAX(election_results.total_votes_secured) - MIN(election_results.total_votes_secured) AS victory_margin
FROM election_results
JOIN candidates ON election_results.candidate_id = candidates.id
WHERE candidates.name = 'Bandi Sanjay Kumar' AND election_results.candidate_position = 1;
""")

# Fetch the result
result = cursor.fetchone()

# Print the result
print(result)

# Close the connection
conn.close()

(0,)


In [94]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT
    er1.total_votes_secured - er2.total_votes_secured AS margin_of_victory
FROM
    election_results AS er1
JOIN
    election_results AS er2 
JOIN
    candidates AS c1 ON er1.candidate_id = c1.id
JOIN
    candidates AS c2 ON er2.candidate_id = c2.id
WHERE
    c1.name = 'Bandi Sanjay Kumar'
    AND c1.constituency_id = c2.constituency_id
    AND er1.candidate_position = 1
    AND er2.candidate_position = 2;
""")

# Fetch the result
result = cursor.fetchall()

# Print the result
print(result)

# Close the connection
conn.close()

[(89508,)]


In [134]:
response = qp.run(query="How many people contested in 2019 LS elections from all the constituencies in Maharashtra?")
print(str(response))

> Running module input with input: 
query: How many people contested in 2019 LS elections from all the constituencies in Maharashtra?

> Running module table_retriever with input: 
input: How many people contested in 2019 LS elections from all the constituencies in Maharashtra?

> Running module table_output_parser with input: 
query_str: How many people contested in 2019 LS elections from all the constituencies in Maharashtra?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='election_stats', context_str='Stores election sta

In [74]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT
    COUNT(DISTINCT c.id) AS num_candidates_in_maharashtra
FROM
    candidates AS c
JOIN
    constituencies AS cn ON c.constituency_id = cn.id
JOIN
    states AS s ON cn.state_id = s.id
JOIN
    elections AS e ON cn.election_id = e.id
WHERE
    e.year = 2019
    AND s.name = 'Maharashtra';
""")

# Fetch the result
result = cursor.fetchone()

# Print the result
print(result)

# Close the connection
conn.close()

(915,)


In [77]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT * FROM candidates AS c
JOIN
    constituencies AS cn ON c.constituency_id = cn.id
JOIN
    states AS s ON cn.state_id = s.id
JOIN
    elections AS e ON cn.election_id = e.id
WHERE
    e.year = 2019
    AND s.name = 'Maharashtra';
""")

# Fetch the result
result = cursor.fetchall()

# Print the result
#print(result)

# Close the connection
conn.close()

In [54]:
response = qp.run(query="In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?")
print(str(response))

> Running module input with input: 
query: In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?

> Running module table_retriever with input: 
input: In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?

> Running module table_output_parser with input: 
query_str: In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='candidates', context_str='Co

In [ ]:
response = qp.run(query="How much percentage of male contesting candidates got their deposits forfeited in Tamil Nadu in 2019?")
print(str(response))

In [100]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT SUM(election_stats.value) AS total_forfeited, COUNT(candidates.id) AS total_candidates
FROM election_stats
JOIN constituencies ON election_stats.constituency_id = constituencies.id
JOIN elections ON constituencies.election_id = elections.id
JOIN candidates ON candidates.constituency_id = constituencies.id
WHERE elections.year = 2019
AND election_stats.category = 'Forfeited Deposits'
AND election_stats.gender = 'Male'
AND constituencies.state_id = (SELECT id FROM states WHERE name = 'Tamil Nadu');
""")

# Fetch the result
result = cursor.fetchone()

# Print the result
print(result)

# Close the connection
conn.close()

(18409, 889)


In [56]:
response = qp.run(query="How many nominations were filed by women in 2019 LS elections in Kozhikode?")
print(str(response))

> Running module input with input: 
query: How many nominations were filed by women in 2019 LS elections in Kozhikode?

> Running module table_retriever with input: 
input: How many nominations were filed by women in 2019 LS elections in Kozhikode?

> Running module table_output_parser with input: 
query_str: How many nominations were filed by women in 2019 LS elections in Kozhikode?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elec

In [ ]:
response = qp.run(query= "In 2019 LS election, in Hyderabad, give me the number of candidates whose deposits were not forfeited? Also, give me their names.")
print(str(response))

In [84]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT candidates.name
FROM candidates
JOIN constituencies ON candidates.constituency_id = constituencies.id
JOIN elections ON constituencies.election_id = elections.id
JOIN election_stats ON constituencies.id = election_stats.constituency_id
WHERE elections.year = 2019 AND constituencies.name = 'Hyderabad' AND election_stats.category = 'Forfeited Deposits' AND election_stats.value = 0;
""")

# Fetch the result
result = cursor.fetchall()

# Print the result
print(result)

# Close the connection
conn.close()

[('Asaduddin Owaisi',), ('Beeramganti Venkat Ramesh Naidu',), ('Dornala Jaya Prakash',), ('Dr. Bhagavanth Rao',), ('Dr. H. Susheel Raj',), ('K. Maheshwar',), ('K. Nagaraj',), ('L. Ashok Nath',), ('Mohammed Abdul Azeem',), ('Mohammed Feroz Khan',), ('Mohd Ahmed',), ('Nota',), ('Pusthe Srikanth',), ('Rangacharya',), ('Sanjay Kumar Shukla',), ('V. Bal Krishna',), ('Asaduddin Owaisi',), ('Beeramganti Venkat Ramesh Naidu',), ('Dornala Jaya Prakash',), ('Dr. Bhagavanth Rao',), ('Dr. H. Susheel Raj',), ('K. Maheshwar',), ('K. Nagaraj',), ('L. Ashok Nath',), ('Mohammed Abdul Azeem',), ('Mohammed Feroz Khan',), ('Mohd Ahmed',), ('Nota',), ('Pusthe Srikanth',), ('Rangacharya',), ('Sanjay Kumar Shukla',), ('V. Bal Krishna',)]


In [89]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('elections.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT
    c.name AS candidate_name,
    COUNT(DISTINCT c.id) AS num_candidates_not_forfeiting_deposit
FROM
    candidates AS c
JOIN
    election_results AS er ON c.id = er.candidate_id
JOIN
    constituencies AS cn ON c.constituency_id = cn.id  
JOIN
    elections AS e ON cn.election_id = e.id
WHERE
    e.year = 2019
    AND cn.name = 'Hyderabad'
    AND er.total_votes_secured > 0;
""")

# Fetch the result
result = cursor.fetchall()

# Print the result
print(result)

# Close the connection
conn.close()   


[('Asaduddin Owaisi', 16)]


In [62]:
response = qp.run(query="Give the names of the parties who lost deposits in South Goa in 2019?")
print(str(response))

> Running module input with input: 
query: Give the names of the parties who lost deposits in South Goa in 2019?

> Running module table_retriever with input: 
input: Give the names of the parties who lost deposits in South Goa in 2019?

> Running module table_output_parser with input: 
query_str: Give the names of the parties who lost deposits in South Goa in 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTa

In [ ]:
response = qp.run(query="Did at least one National party lost deposit in Nalgonda in 2019?")
print(str(response))

In [60]:
response = qp.run(query="Did any woman lost deposit in Nizamabad in 2019? If yes, give names.")
print(str(response))

> Running module input with input: 
query: Did any woman lost deposit in Nizamabad in 2019? If yes, give names.

> Running module table_retriever with input: 
input: Did any woman lost deposit in Nizamabad in 2019? If yes, give names.

> Running module table_output_parser with input: 
query_str: Did any woman lost deposit in Nizamabad in 2019? If yes, give names.
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes th

In [63]:
response = qp.run(query="Who won in the Kukatpally consituency in the 2014 elections?")
print(str(response))

> Running module input with input: 
query: Who won in the Kukatpally consituency in the 2014 elections?

> Running module table_retriever with input: 
input: Who won in the Kukatpally consituency in the 2014 elections?

> Running module table_output_parser with input: 
query_str: Who won in the Kukatpally consituency in the 2014 elections?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, categ

In [64]:
response = qp.run(query="How many votes did Narendra Modi get in 2009 elections?")
print(str(response))

> Running module input with input: 
query: How many votes did Narendra Modi get in 2009 elections?

> Running module table_retriever with input: 
input: How many votes did Narendra Modi get in 2009 elections?

> Running module table_output_parser with input: 
query_str: How many votes did Narendra Modi get in 2009 elections?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding v

In [65]:
response = qp.run(query="Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?")
print(str(response))

> Running module input with input: 
query: Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?

> Running module table_retriever with input: 
input: Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?

> Running module table_output_parser with input: 
query_str: Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='constituencies', context_str='Contains inform

In [66]:
response = qp.run(query="How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?")
print(str(response))

> Running module input with input: 
query: How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?

> Running module table_retriever with input: 
input: How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?

> Running module table_output_parser with input: 
query_str: How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id

In [65]:
response = qp.run(query="Asaduddin Owaisi")
print(str(response))

> Running module input with input: 
query: Asaduddin Owaisi

> Running module table_retriever with input: 
input: Asaduddin Owaisi

> Running module table_output_parser with input: 
query_str: Asaduddin Owaisi
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in e

In [66]:
response = qp.run(query="Asaduddin")
print(str(response))

> Running module input with input: 
query: Asaduddin

> Running module table_retriever with input: 
input: Asaduddin

> Running module table_output_parser with input: 
query_str: Asaduddin
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participat

In [67]:
response = qp.run(query="Owaisi")
print(str(response))

> Running module input with input: 
query: Owaisi

> Running module table_retriever with input: 
input: Owaisi

> Running module table_output_parser with input: 
query_str: Owaisi
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the cand

In [68]:
response = qp.run(query="Asadudin Owaesi")
print(str(response))

> Running module input with input: 
query: Asadudin Owaesi

> Running module table_retriever with input: 
input: Asadudin Owaesi

> Running module table_output_parser with input: 
query_str: Asadudin Owaesi
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elec

In [69]:
response = qp.run(query="How many members are contested in the AP in 2019?")
print(str(response))

> Running module input with input: 
query: How many members are contested in the AP in 2019?

> Running module table_retriever with input: 
input: How many members are contested in the AP in 2019?

> Running module table_output_parser with input: 
query_str: How many members are contested in the AP in 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the correspondin

In [67]:
response = qp.run(query="Which party won the most seats in Uttar Pradesh in 2019?")
print(str(response))

> Running module input with input: 
query: Which party won the most seats in Uttar Pradesh in 2019?

> Running module table_retriever with input: 
input: Which party won the most seats in Uttar Pradesh in 2019?

> Running module table_output_parser with input: 
query_str: Which party won the most seats in Uttar Pradesh in 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQL

In [69]:
response = qp.run(query="What was the total number of votes cast in the Mumbai North constituency in 2019?")
print(str(response))

> Running module input with input: 
query: What was the total number of votes cast in the Mumbai North constituency in 2019?

> Running module table_retriever with input: 
input: What was the total number of votes cast in the Mumbai North constituency in 2019?

> Running module table_output_parser with input: 
query_str: What was the total number of votes cast in the Mumbai North constituency in 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the a

In [71]:
response = qp.run(query="Who was the youngest candidate to win in the 2019 elections?")
print(str(response))

> Running module input with input: 
query: Who was the youngest candidate to win in the 2019 elections?

> Running module table_retriever with input: 
input: Who was the youngest candidate to win in the 2019 elections?

> Running module table_output_parser with input: 
query_str: Who was the youngest candidate to win in the 2019 elections?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position

In [73]:
response = qp.run(query="Which constituency had the lowest voter turnout in 2019?")
print(str(response))

> Running module input with input: 
query: Which constituency had the lowest voter turnout in 2019?

> Running module table_retriever with input: 
input: Which constituency had the lowest voter turnout in 2019?

> Running module table_output_parser with input: 
query_str: Which constituency had the lowest voter turnout in 2019?
table_schema_objs: [SQLTableSchema(table_name='constituencies', context_str='Contains information about election constituencies, including the associated state_id,elction_id, unique constituency number, constituency name, and constituency type (e.g., General, Scheduled Caste, Scheduled Tribe).'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='elections', context_str='Holds details about loksab

In [74]:
response = qp.run(query="What was the average age of candidates in the 2019 Lok Sabha elections?")
print(str(response))

> Running module input with input: 
query: What was the average age of candidates in the 2019 Lok Sabha elections?

> Running module table_retriever with input: 
input: What was the average age of candidates in the 2019 Lok Sabha elections?

> Running module table_output_parser with input: 
query_str: What was the average age of candidates in the 2019 Lok Sabha elections?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the correspo

In [75]:
response = qp.run(query="Which state had the highest number of female candidates in 2019?")
print(str(response))

> Running module input with input: 
query: Which state had the highest number of female candidates in 2019?

> Running module table_retriever with input: 
input: Which state had the highest number of female candidates in 2019?

> Running module table_output_parser with input: 
query_str: Which state had the highest number of female candidates in 2019?
table_schema_objs: [SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique I

In [76]:
response = qp.run(query="How many candidates contested for the first time in the 2019 elections?")
print(str(response))

> Running module input with input: 
query: How many candidates contested for the first time in the 2019 elections?

> Running module table_retriever with input: 
input: How many candidates contested for the first time in the 2019 elections?

> Running module table_output_parser with input: 
query_str: How many candidates contested for the first time in the 2019 elections?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the correspo

In [77]:
response = qp.run(query="Which party had the highest vote share in Kerala in 2019?")
print(str(response))

> Running module input with input: 
query: Which party had the highest vote share in Kerala in 2019?

> Running module table_retriever with input: 
input: Which party had the highest vote share in Kerala in 2019?

> Running module table_output_parser with input: 
query_str: Which party had the highest vote share in Kerala in 2019?
table_schema_objs: [SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='parties', context_str='Stores information about political parties participating in the elections. It includes the party ID, party name, abbreviation, and party type.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), 

In [78]:
response = qp.run(query="Who was the oldest candidate to contest in the 2019 elections?")
print(str(response))

> Running module input with input: 
query: Who was the oldest candidate to contest in the 2019 elections?

> Running module table_retriever with input: 
input: Who was the oldest candidate to contest in the 2019 elections?

> Running module table_output_parser with input: 
query_str: Who was the oldest candidate to contest in the 2019 elections?
table_schema_objs: [SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statis

In [79]:
response = qp.run(query="What was the average margin of victory across all constituencies in 2019?")
print(str(response))

> Running module input with input: 
query: What was the average margin of victory across all constituencies in 2019?

> Running module table_retriever with input: 
input: What was the average margin of victory across all constituencies in 2019?

> Running module table_output_parser with input: 
query_str: What was the average margin of victory across all constituencies in 2019?
table_schema_objs: [SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value for each statistic. This table provides aggregated insights into election outcomes.'), SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQLTableSchema(table_name='constituencies', cont

In [80]:
response = qp.run(query="Which candidate secured the highest percentage of votes in their constituency in 2019?")
print(str(response))

> Running module input with input: 
query: Which candidate secured the highest percentage of votes in their constituency in 2019?

> Running module table_retriever with input: 
input: Which candidate secured the highest percentage of votes in their constituency in 2019?

> Running module table_output_parser with input: 
query_str: Which candidate secured the highest percentage of votes in their constituency in 2019?
table_schema_objs: [SQLTableSchema(table_name='election_results', context_str="Records the election results, including candidate ID, general votes secured, postal votes secured, total votes, percentage of votes secured, and the candidate's position in the election results."), SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, ca

In [81]:
response = qp.run(query="How many candidates in the 2019 elections were below the age of 30?")
print(str(response))

> Running module input with input: 
query: How many candidates in the 2019 elections were below the age of 30?

> Running module table_retriever with input: 
input: How many candidates in the 2019 elections were below the age of 30?

> Running module table_output_parser with input: 
query_str: How many candidates in the 2019 elections were below the age of 30?
table_schema_objs: [SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value 

In [139]:
response = qp.run(query="How many candidates in the 2019 elections were below the age of 30?")
print(str(response))

> Running module input with input: 
query: How many candidates in the 2019 elections were below the age of 30?

> Running module table_retriever with input: 
input: How many candidates in the 2019 elections were below the age of 30?

> Running module table_output_parser with input: 
query_str: How many candidates in the 2019 elections were below the age of 30?
table_schema_objs: [SQLTableSchema(table_name='candidates', context_str='Contains details about candidates contesting in elections. It includes the candidate ID, name, gender, age, category, associated constituency_id, and party_id.'), SQLTableSchema(table_name='elections', context_str='Holds details about loksabha elections, including their unique ID, type (e.g.,loksabha), year of occurrence, and a unique election number.'), SQLTableSchema(table_name='election_stats', context_str='Stores election statistics categorized by gender, category, and value. It includes the constituency ID, category, gender, and the corresponding value 

In [119]:
#Applying the RetryAgent

In [140]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=['states','constituencies','elections','parties','candidates','election_results','election_stats'],
    verbose=True,
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

In [141]:
from llama_index.core.agent import Task, AgentChatResponse
from typing import Dict, Any
from llama_index.core.query_pipeline import (
    AgentInputComponent,
    AgentFnComponent,
)


def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict:
    """Agent input function."""
    # initialize current_reasoning
    if "convo_history" not in state:
        state["convo_history"] = []
        state["count"] = 0
    state["convo_history"].append(f"User: {task.input}")
    convo_history_str = "\n".join(state["convo_history"]) or "None"
    return {"input": task.input, "convo_history": convo_history_str}


agent_input_component = AgentInputComponent(fn=agent_input_fn)

In [142]:
from llama_index.core import PromptTemplate

retry_prompt_str = """\
You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent which
will convert the query to a SQL statement. If the agent triggers an error,
then that will be reflected in the current conversation history (see below).

If the conversation history is None, use the user input. If its not None,
generate a new SQL query that avoids the problems of the previous SQL query.

Input: {input}
Convo history (failed attempts):
{convo_history}

New input: """
retry_prompt = PromptTemplate(retry_prompt_str)

In [143]:
from llama_index.core import Response
from typing import Tuple

validate_prompt_str = """\
Given the user query, validate whether the inferred SQL query and response from executing the query is correct and answers the query.

Answer with YES or NO.

Query: {input}
Inferred SQL query: {sql_query}
SQL Response: {sql_response}

Result: """
validate_prompt = PromptTemplate(validate_prompt_str)

MAX_ITER = 3


def agent_output_fn(
    task: Task, state: Dict[str, Any], output: Response
) -> Tuple[AgentChatResponse, bool]:
    """Agent output component."""
    print(f"> Inferred SQL Query: {output.metadata['sql_query']}")
    print(f"> SQL Response: {str(output)}")
    state["convo_history"].append(
        f"Assistant (inferred SQL query): {output.metadata['sql_query']}"
    )
    state["convo_history"].append(f"Assistant (response): {str(output)}")

    # run a mini chain to get response
    validate_prompt_partial = validate_prompt.as_query_component(
        partial={
            "sql_query": output.metadata["sql_query"],
            "sql_response": str(output),
        }
    )
    qp = QP(chain=[validate_prompt_partial, llm])
    validate_output = qp.run(input=task.input)

    state["count"] += 1
    is_done = False
    if state["count"] >= MAX_ITER:
        is_done = True
    if "YES" in validate_output.message.content:
        is_done = True

    return AgentChatResponse(response=str(output)), is_done


agent_output_component = AgentFnComponent(fn=agent_output_fn)

In [144]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)

qp = QP(
    modules={
        "input": agent_input_component,
        "retry_prompt": retry_prompt,
        "llm": llm,
        "sql_query_engine": sql_query_engine,
        "output_component": agent_output_component,
    },
    verbose=True,
)
qp.add_link("input", "retry_prompt", src_key="input", dest_key="input")
qp.add_link(
    "input", "retry_prompt", src_key="convo_history", dest_key="convo_history"
)
qp.add_chain(["retry_prompt", "llm", "sql_query_engine", "output_component"])

In [145]:
from llama_index.core.agent import QueryPipelineAgentWorker, AgentRunner
from llama_index.core.callbacks import CallbackManager

# callback manager is passed from query pipeline to agent worker/agent
agent_worker = QueryPipelineAgentWorker(qp)
agent1 = AgentRunner(
    agent_worker, callback_manager=CallbackManager(), verbose=False
)

In [147]:
response = agent1.chat(
    "Who got the highest number of votes in 2019 in Hyderabad?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='5d862f78-cf59-4ea7-994e-59b61bb58823' input='Who got the highest number of votes in 2019 in Hyderabad?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Who got the highest number of votes in 2019 in Hyderabad?
convo_history: User: Who got the highest number of votes in 2019 in Hyderabad?

> Running module llm with input: 
messages: You are 

In [170]:
response = agent1.chat(
    "How many Lok Sabha constituencies are there in Andhra Pradesh?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='d662e6e9-75c0-48c2-a656-89ebfef06f45' input='How many Lok Sabha constituencies are there in Andhra Pradesh?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How many Lok Sabha constituencies are there in Andhra Pradesh?
convo_history: User: How many Lok Sabha constituencies are there in Andhra Pradesh?

> Running module llm with input: 
mes

In [171]:
response = agent1.chat(
    "How many ST candidates contested from BJP in Chhattisgarh in 2014?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='ab8131c4-f39e-4320-8003-5cca107015c4' input='How many ST candidates contested from BJP in Chhattisgarh in 2014?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How many ST candidates contested from BJP in Chhattisgarh in 2014?
convo_history: User: How many ST candidates contested from BJP in Chhattisgarh in 2014?

> Running module llm with

In [172]:
response = agent1.chat(
    "What was the margin of victory for Bandi Sanjay Kumar?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='e1f8cb62-2738-4a39-ae34-ef4fea720d7d' input='What was the margin of victory for Bandi Sanjay Kumar?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: What was the margin of victory for Bandi Sanjay Kumar?
convo_history: User: What was the margin of victory for Bandi Sanjay Kumar?

> Running module llm with input: 
messages: You are trying to

In [174]:
response = agent1.chat(
    "How much percentage of male contesting candidates got their deposits forfeited in Tamil Nadu in 2019?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='3bed6648-6420-46b4-a2d4-ce64ce915245' input='How much percentage of male contesting candidates got their deposits forfeited in Tamil Nadu in 2019?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How much percentage of male contesting candidates got their deposits forfeited in Tamil Nadu in 2019?
convo_history: User: How much percentage of 

In [176]:
response = agent1.chat(
    "How many people contested in 2019 LS elections from all the constituencies in Maharashtra?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='015a635a-0b69-4a3f-8e51-bf94a6bc3d8a' input='How many people contested in 2019 LS elections from all the constituencies in Maharashtra?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How many people contested in 2019 LS elections from all the constituencies in Maharashtra?
convo_history: User: How many people contested in 2019 LS election

In [154]:
response = agent1.chat(
    "In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='21352b2c-762a-4174-b9bb-cf7d53a0fc0a' input='In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: In which constituency of Delhi, the highest numbers of candidates contested in the 2014 election?
convo_history: User: In which constituency of Delhi,

In [175]:
response = agent1.chat(
    "How many nominations were filed by women in 2019 LS elections in Kozhikode?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='c5526d5f-5cff-4e4e-9527-92f439012940' input='How many nominations were filed by women in 2019 LS elections in Kozhikode?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How many nominations were filed by women in 2019 LS elections in Kozhikode?
convo_history: User: How many nominations were filed by women in 2019 LS elections in Kozhikode?

In [177]:
response = agent1.chat(
    "In 2019 LS election, in Hyderabad, give me the number of candidates whose deposits were not forfeited? Also, give me their names."
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='22e4625a-358e-4c72-8015-81f36a30cbd9' input='In 2019 LS election, in Hyderabad, give me the number of candidates whose deposits were not forfeited? Also, give me their names.' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: In 2019 LS election, in Hyderabad, give me the number of candidates whose deposits were not forfeited? Also, give me t

In [178]:
response = agent1.chat(
    "Give the names of the parties who lost deposits in South Goa in 2019?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='4e285d37-33e7-4dea-a284-d94a89ba1f0c' input='Give the names of the parties who lost deposits in South Goa in 2019?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Give the names of the parties who lost deposits in South Goa in 2019?
convo_history: User: Give the names of the parties who lost deposits in South Goa in 2019?

> Running module

In [179]:
response = agent1.chat(
    "Did at least one National party lost deposit in Nalgonda in 2019?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='1b85b219-5740-45e0-ba9b-1b592548059d' input='Did at least one National party lost deposit in Nalgonda in 2019?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Did at least one National party lost deposit in Nalgonda in 2019?
convo_history: User: Did at least one National party lost deposit in Nalgonda in 2019?

> Running module llm with in

In [180]:
response = agent1.chat(
    "Did any woman lost deposit in Nizamabad in 2019? If yes, give names."
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='6f4fe8c9-e61f-455e-9232-0348679e6320' input='Did any woman lost deposit in Nizamabad in 2019? If yes, give names.' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Did any woman lost deposit in Nizamabad in 2019? If yes, give names.
convo_history: User: Did any woman lost deposit in Nizamabad in 2019? If yes, give names.

> Running module ll

In [181]:
response = agent1.chat(
    "Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='ccb3e01f-b7ff-470a-ae6b-0e3f5d4a8592' input='Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Niranjan Jyoti defeated Raj Babbar in Fatehpur constituency in 2019 by how many votes?
convo_history: User: Niranjan Jyoti defeated Raj Babbar in Fatehpur consti

In [182]:
response = agent1.chat(
    "In which consituency NOTA came in the first position?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='de900d0a-1a1a-4787-ac14-46b7c8735fb4' input='In which consituency NOTA came in the first position?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: In which consituency NOTA came in the first position?
convo_history: User: In which consituency NOTA came in the first position?

> Running module llm with input: 
messages: You are trying to ge

In [183]:
response = agent1.chat(
    "How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='d2254ea8-38b1-4876-a8b8-a1e66ee2767f' input='How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How many candidates contested in the Amravati constituency of Andhra Pradesh in 2019?
convo_history: User: How many candidates contested in the Amravati constitue

In [163]:
response = agent1.chat(
    "Who won in the Kukatpally consituency in the 2014 elections?"
)
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='ef38e1e2-a352-4689-90de-9fa6e86edf18' input='Who won in the Kukatpally consituency in the 2014 elections?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Who won in the Kukatpally consituency in the 2014 elections?
convo_history: User: Who won in the Kukatpally consituency in the 2014 elections?

> Running module llm with input: 
messages:

In [164]:
response = agent1.chat(
    "Asaduddin Owaisi")
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='fc5a0423-4290-4bcf-9ae9-cfc476a2834b' input='Asaduddin Owaisi' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Asaduddin Owaisi
convo_history: User: Asaduddin Owaisi

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream

In [165]:
response = agent1.chat(
    "Asaduddin")
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='7ef82dab-9683-42f1-8801-56c4db5cbcad' input='Asaduddin' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Asaduddin
convo_history: User: Asaduddin

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent wh

In [166]:
response = agent1.chat(
    "Owaisi")
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='78def515-ab91-4e44-b3fd-21c2d75d6910' input='Owaisi' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Owaisi
convo_history: User: Owaisi

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream text-to-SQL agent which
will 

In [167]:
#Giving candidate name in Capital Letters
response = agent1.chat(
    "ASADUDDIN OWAISI")
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='4b109a72-4c8c-4f8e-abb3-860386aab35d' input='ASADUDDIN OWAISI' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: ASADUDDIN OWAISI
convo_history: User: ASADUDDIN OWAISI

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream

In [168]:
#Incorrect spelling of a candidate
response = agent1.chat(
    "Asadudin Owaesi")
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='68cd5e63-7d6e-4ae8-a4dc-1b5b9500107f' input='Asadudin Owaesi' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: Asadudin Owaesi
convo_history: User: Asadudin Owaesi

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query will then be interpreted by a downstream te

In [169]:
response = agent1.chat(
    "How many contested in the AP?")
print(str(response))

> Running module input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={}), chat_store_key='chat_history')}
task: task_id='a0ec2a19-99bb-46d0-b783-ba23224f8ca6' input='How many contested in the AP?' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatStore(store={'chat_history': []}), chat_store_key='chat_history') callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f586dbe2bc0> extra_state={}

> Running module retry_prompt with input: 
input: How many contested in the AP?
convo_history: User: How many contested in the AP?

> Running module llm with input: 
messages: You are trying to generate a proper natural language query given a user input.

This query w